In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import requests
import traceback
import pandas as pd

Pop up browser Chrome

In [5]:
# Wajib dijalanin
driver = webdriver.Chrome()
driver.get("https://enrichment.apps.binus.ac.id/Login") 

In [4]:
# Kalau mau automated login, isi email dan password, kalau tidak, biarkan kosong trus login manual via browser pop up
email = ""
password = ""

Automated Login w/ Selenium

In [6]:
login_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="header"]/div/section[2]/nav/div/ul[2]/a'))
                )
login_button.click()

In [7]:
microsoft_sign_in_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="btnLogin"]'))
                )
microsoft_sign_in_button.click()

In [9]:
# login with microsoft account
driver.find_element(By.CSS_SELECTOR, 'input[name="loginfmt"]').send_keys(email)

In [10]:
next_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="idSIButton9"]'))
                )
next_button.click()

In [11]:
driver.find_element(By.CSS_SELECTOR, 'input[name="passwd"]').send_keys(password)

In [13]:
sign_in_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="idSIButton9"]'))
                )
sign_in_button.click()

In [14]:
yes_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="idSIButton9"]'))
                )
yes_button.click()

In [15]:
activity_enrichment_app = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="StudentTermDashboard"]/span[1]/a[2]'))
                )
activity_enrichment_app.click()

In [16]:
select_account = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="tilesHolder"]/div[1]/div/div[1]'))
                )
select_account.click()

Get cookie, then send request payload

In [18]:
session = requests.Session()
for cookie in driver.get_cookies():
    print({cookie['name']: cookie['value']})
    session.cookies.update({cookie['name']: cookie['value']})

print(session.cookies)

{'_ga': 'GA1.3.1377940165.1715250874'}
{'_ga_6QNPVQ9YZ8': 'GS1.1.1715250953.1.0.1715250953.0.0.0'}
{'.AspNetCore.Antiforgery.9fXoN5jHCXs': 'CfDJ8EJ-S0CFkzdKknk1Ofb49PzzYmtq9BaHd_id2tgaaWkXxAeAqyPACaju5ATlMdvjIBhX6vu9cEPtAfmJYfaPO7mhGswZpm80qHdfpN3G-Yg8jZd6jE79X4S0mV209oEjH9RSDz5IwL2F1LvHjJgPiKs'}
{'.BinusActivity.Session': 'CfDJ8EJ%2BS0CFkzdKknk1Ofb49PwGofEjjRd%2BBA2phTKywcIhRcXlYhKcrsTEawGdefnfBTH8INfJLZk9%2B4gpm2U%2Bs%2BUSSdlj8fB1M4uNMiDwBC9AEK6dP0GxgSMPrD1qi2v1EmmWk0dGFPNObkV5ghzXhBhGT5F6E9Ix74QslBEylEoa'}
{'_gat_gtag_UA_178239469_8': '1'}
{'.AspNetCore.Cookies': 'CfDJ8EJ-S0CFkzdKknk1Ofb49Px_h5KNHsaISrAGYGZumvW5wOjnQMs2e13qWkwe3ms3X9k1-AHiSiao3E3emINYYvJ9paharhrmta9onJf7itk4f_SX2KWyl15fRuEzSYwquh-D6g09yNc-sItMvQ39Y1V6T_VzRp-FmdZKm_cjSiIVNFHZoVz2Fq_jqwzQEObCOab4z5LCa5wlFPtFaKgXGtvDv6VBnmRhIlCeCJ8UBqV2QP34ztSHYJbPUoswvtkau-R1YGdsf_nZSSFsJEuPLVJm-NnSc6pCiGi6EfbAFDFiV_0MsAa35ZLRhNFcHBkjxFJPimI5oxFG4BnI1o1PMAumqlfD754JJxBzXWwIdXpVc29PI9D31n12upL0CeZkDAuaOiFRblh_S6FM79b39AwfsyuKrPNWIGbkfl

In [19]:
data = pd.read_csv("Sunib logger - April.csv", dtype = 'string', keep_default_na = False) # Ganti nama file csv sesuai dengan nama file csv yang diinginkan
base_url = "https://activity-enrichment.apps.binus.ac.id/LogBook"

logbook_id_mapping = {}

In [20]:
def get_months():
	try:
		response = session.get(base_url + "/GetMonths")
		raw_month_mapping = response.json()

		if response.status_code != 200:
			raise Exception(f"{response.text}")

		for month in raw_month_mapping['data']:
			logbook_id_mapping.update({
				f"{month['monthInt']:02d}" : f"{month['logBookHeaderID']}"
			})
	except Exception:
		exception_message = traceback.format_exc()
		print(f"Failed to get logbook ID. Reason: {exception_message}")

In [21]:
def send_request(date, clock_in, clock_out, activity, description):
	try:
		payload = {
			'model[ID]': "00000000-0000-0000-0000-000000000000",
			'model[LogBookHeaderID]': logbook_id_mapping[date[5:7]],
			'model[Date]': f"{date}T00:00:00",
			'model[Activity]': activity,
			'model[ClockIn]': clock_in,
			'model[ClockOut]': clock_out,
			'model[Description]': description
		}
		response = session.post(base_url + "/StudentSave", data = payload)
		print(f"{date}: {response.text}")
	except Exception:
		exception_message = traceback.format_exc()
		print(f"{date}: {exception_message}")

In [22]:
def main():
	get_months()

	for key, log in data.iterrows():
		if log['activity'] == "":
			continue
		elif log['activity'] == "OFF":
			send_request(str(log['date']), "OFF", "OFF", "OFF", "OFF")
		else:
			send_request(str(log['date']), "09:00 am", "06:00 pm", log['activity'], log['description'])

In [23]:
main()

2024-04-01: {"json":true,"status":"Successfully added a new record!"}
2024-04-02: {"json":true,"status":"Successfully added a new record!"}
2024-04-03: {"json":true,"status":"Successfully added a new record!"}
2024-04-04: {"json":true,"status":"Successfully added a new record!"}
2024-04-05: {"json":true,"status":"Successfully added a new record!"}
2024-04-06: {"json":true,"status":"Successfully added a new record!"}
2024-04-07: {"json":true,"status":"Successfully added a new record!"}
2024-04-08: {"json":true,"status":"Successfully added a new record!"}
2024-04-09: {"json":true,"status":"Successfully added a new record!"}
2024-04-10: {"json":true,"status":"Successfully added a new record!"}
2024-04-11: {"json":true,"status":"Successfully added a new record!"}
2024-04-12: {"json":true,"status":"Successfully added a new record!"}
2024-04-13: {"json":true,"status":"Successfully added a new record!"}
2024-04-14: {"json":true,"status":"Successfully added a new record!"}
2024-04-15: {"json":